In [1]:
import numpy as np
import pandas as pd

import sklearn.metrics
import sklearn.datasets
import sklearn.model_selection

import sklearn.tree
import sklearn.linear_model

import matplotlib.pyplot as plt
%matplotlib inline

# Реализация функции

In [2]:
def stack_pred(estimator,X,y,Xt,yt,k,method):
    kf = sklearn.model_selection.KFold(n_splits=k, shuffle=True, random_state=0)
    
    ## Определим функцию голосования (в спорных моментах выигрывает класс имеющий наименьшее значение, то есть из [0,2,3] выигрывает 0)
    def voting(x,classes):
        buf = []
        for row in x:
            #print(row)
            dct={}
            for clas in classes:
                dct[clas]=0
            for i in row:
                dct[i]+=1
            #print(dct)
            #print(max(dct,key=dct.get))
            buf.append(max(dct,key=dct.get))
        return np.array(buf,dtype='int32')


    
    ## Инициализируем стэкинги нулями, с размерностями, соответсвующими методам.
    ## То есть одномерный вектор для predict и матрица вероятностей для predict_proba
    
    if method == 'predict':
        sX=np.zeros(shape=(X.shape[0]),dtype='int32')
        sXt=np.zeros(shape=(Xt.shape[0]),dtype='int32')
        vote=np.zeros(shape=(Xt.shape[0]),dtype='int32')

    elif method == 'predict_proba':
        sX=np.zeros(shape=(X.shape[0],np.unique(y).shape[0]),dtype='float32')
        sXt=np.zeros(shape=(Xt.shape[0],np.unique(y).shape[0]),dtype='float32')
        vote=np.zeros(shape=(Xt.shape[0],np.unique(y).shape[0]),dtype='float32')

    else:
        print('Incorrect method!')
        return -1
    
    
    ## Осуществляем кросс-валидацию
    
    scores=[]
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        estimator.fit(X_train, y_train)
        
        if method == 'predict':
            pred = estimator.predict(X_test)
            predt = estimator.predict(Xt)
            vote = np.concatenate([vote,predt])
            scores.append(sklearn.metrics.accuracy_score(yt, predt))

        elif method == 'predict_proba':
            pred = estimator.predict_proba(X_test)
            predt = estimator.predict_proba(Xt)
            vote += predt
        
        ## Записваем результаты в наш трэйн-стэкинг.
        sX[test_index]=pred
    
    
    if method == 'predict': ## осущесвляем голосование по принцпу "кто больше"
        #print(scores)
        vote = vote.reshape(k+1,yt.shape[0]) 
        vote = np.transpose(vote[1:])
        vote = voting(vote,np.unique(y))
        
    elif method == 'predict_proba': ## смотрим среднее арифмитеческое вероятностей
        vote/=k
        
    
    ## Записваем результаты в наш тест-стэкинг.       
    sXt=vote
    
    print('Done!')
    
    return sX,sXt

# Пример

Загрузка и разбиение данных:

In [3]:
X_full, y_full = sklearn.datasets.load_breast_cancer(return_X_y=True)

In [4]:
X, Xt, y, yt = sklearn.model_selection.train_test_split(X_full, y_full, test_size=0.25, random_state=0)

Рассмотрим в качестве примеров эстиматоров решающее дерево и логистическую регрессию.

In [5]:
clf_lr = sklearn.linear_model.LogisticRegression(solver='lbfgs',max_iter=3000)
clf_dt = sklearn.tree.DecisionTreeClassifier(random_state=0, min_samples_split=8, min_samples_leaf=4)

#### method = 'predict'

In [6]:
sx_dt,sxt_dt=stack_pred(clf_dt,X,y,Xt,yt,5,'predict')
sx_lr,sxt_lr=stack_pred(clf_lr,X,y,Xt,yt,5,'predict')
print(sx_dt.shape,sxt_dt.shape)
print(sx_dt[5:10])
print(sxt_dt[5:10])

Done!
Done!
(426,) (143,)
[1 1 1 0 1]
[1 1 1 1 1]


Сравним accuracy

In [7]:
clf_dt.fit(X,y)
pr=clf_dt.predict(Xt)
print('DT: С использоваеним функции:', sklearn.metrics.accuracy_score(yt, sxt_dt))
print('DT: Без использования:',sklearn.metrics.accuracy_score(yt, pr))
print()

clf_lr.fit(X,y)
pr=clf_lr.predict(Xt)
print('LR: С использоваеним функции:', sklearn.metrics.accuracy_score(yt, sxt_lr))
print('LR: Без использования:',sklearn.metrics.accuracy_score(yt, pr))

DT: С использоваеним функции: 0.965034965034965
DT: Без использования: 0.916083916083916

LR: С использоваеним функции: 0.951048951048951
LR: Без использования: 0.951048951048951


In [33]:
X_new=pd.DataFrame(np.transpose(np.concatenate([sx_dt,sx_lr]).reshape(2,426)))
Xt_new=pd.DataFrame(np.transpose(np.concatenate([sxt_dt,sxt_lr]).reshape(2,143)))

In [34]:
clf = sklearn.tree.K(random_state=0)

In [35]:
clf.fit(X_new,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [39]:
pr=clf.predict(Xt_new)
print('DT: С использоваеним функции:', sklearn.metrics.accuracy_score(yt, pr))

DT: С использоваеним функции: 0.951048951048951


#### method = 'predict_proba'

In [608]:
sx_dt,sxt_dt=stack_pred(clf_dt,X,y,Xt,yt,5,'predict_proba')
sx_lr,sxt_lr=stack_pred(clf_lr,X,y,Xt,yt,5,'predict_proba')
print(sx_dt.shape,sxt_dt.shape)
print(sx_dt[5:10])
print(sxt_dt[:5])

Done!
Done!
(426, 2) (143, 2)
[[0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [1.   0.  ]
 [0.25 0.75]]
[[1.         0.        ]
 [0.18571427 0.81428564]
 [0.         1.        ]
 [0.05       0.95      ]
 [0.         1.        ]]


Сравним log_loss

In [609]:
clf_dt.fit(X,y)
pr=clf_dt.predict_proba(Xt)
print('DT: С использоваеним функции:', sklearn.metrics.log_loss(yt, sxt_dt))
print('DT: Без использования:',sklearn.metrics.log_loss(yt, pr))
print()

clf_lr.fit(X,y)
pr=clf_lr.predict_proba(Xt)
print('LR: С использоваеним функции:', sklearn.metrics.log_loss(yt, sxt_lr))
print('LR: Без использования:',sklearn.metrics.log_loss(yt, pr))

DT: С использоваеним функции: 0.08758764833837122
DT: Без использования: 1.0657655994422166

LR: С использоваеним функции: 0.10781664345334678
LR: Без использования: 0.10668813181332457


## Конец

In [77]:
def stack_pred(estimator,X,y,Xt,yt,k,method):
    kf = sklearn.model_selection.KFold(n_splits=k, shuffle=True, random_state=0)
    
    ## Определим функцию голосования (в спорных моментах выигрывает класс имеющий наименьшее значение, то есть из [0,2,3] выигрывает 0)
    def voting(x,classes):
        buf = []
        for row in x:
            #print(row)
            dct={}
            for clas in classes:
                dct[clas]=0
            for i in row:
                dct[i]+=1
            #print(dct)
            #print(max(dct,key=dct.get))
            buf.append(max(dct,key=dct.get))
        return np.array(buf,dtype='int32')


    
    ## Инициализируем стэкинги нулями, с размерностями, соответсвующими методам.
    ## То есть одномерный вектор для predict и матрица вероятностей для predict_proba
    
    if method == 'predict':
        sX=np.zeros(shape=(X.shape[0]),dtype='int32')
        sXt=np.zeros(shape=(Xt.shape[0]),dtype='int32')
        vote=np.zeros(shape=(Xt.shape[0]),dtype='int32')

    elif method == 'predict_proba':
        sX=np.zeros(shape=(X.shape[0],np.unique(y).shape[0]),dtype='float32')
        sXt=np.zeros(shape=(Xt.shape[0],np.unique(y).shape[0]),dtype='float32')
        vote=np.zeros(shape=(Xt.shape[0],np.unique(y).shape[0]),dtype='float32')
        vote2=np.zeros(shape=(Xt.shape[0],np.unique(y).shape[0]),dtype='float32')



    else:
        print('Incorrect method!')
        return -1
    
    
    ## Осуществляем кросс-валидацию
    
    scores=[]
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        estimator.fit(X_train, y_train)
        
        if method == 'predict':
            pred = estimator.predict(X_test)
            predt = estimator.predict(Xt)
            vote = np.concatenate([vote,predt])
            scores.append(sklearn.metrics.accuracy_score(yt, predt))

        elif method == 'predict_proba':
            pred = estimator.predict_proba(X_test)
            predt = estimator.predict_proba(Xt)
            vote += predt
            vote2 += predt*sklearn.metrics.log_loss(yt, predt)
            scores.append(sklearn.metrics.log_loss(yt, predt))
        
        ## Записваем результаты в наш трэйн-стэкинг.
        sX[test_index]=pred
    
    
    if method == 'predict': ## осущесвляем голосование по принцпу "кто больше"
        scores=np.array(scores)
        scores=scores/np.sum(scores)
        print(scores)
        vote = vote.reshape(k+1,yt.shape[0]) 
        vote = np.transpose(vote[1:])
        vote = voting(vote,np.unique(y))
        
    elif method == 'predict_proba': ## смотрим среднее арифмитеческое вероятностей
        scores=np.array(scores)
        print(scores)
        vote/=k
        #vote=vote2/np.sum(scores)
    
    ## Записваем результаты в наш тест-стэкинг.       
    sXt=vote
    
    print('Done!')
    
    return sX,sXt

In [78]:
sc=[0.19822485, 0.20266272, 0.19822485, 0.20266272, 0.19822485]

In [79]:
X_full, y_full = sklearn.datasets.load_breast_cancer(return_X_y=True)

In [80]:
X, Xt, y, yt = sklearn.model_selection.train_test_split(X_full, y_full, test_size=0.25, random_state=0)

Рассмотрим в качестве примеров эстиматоров решающее дерево и логистическую регрессию.

In [81]:
clf_lr = sklearn.linear_model.LogisticRegression(solver='lbfgs',max_iter=3000)
clf_dt = sklearn.tree.DecisionTreeClassifier(random_state=0, min_samples_split=8, min_samples_leaf=4)

#### method = 'predict_proba'

In [82]:
sx_dt,sxt_dt=stack_pred(clf_dt,X,y,Xt,yt,5,'predict_proba')
sx_lr,sxt_lr=stack_pred(clf_lr,X,y,Xt,yt,5,'predict_proba')
print(sx_dt.shape,sxt_dt.shape)
print(sx_dt[5:10])
print(sxt_dt[:5])

[1.00206166 0.30328071 1.50872147 0.75568458 1.51493347]
Done!
[0.10252823 0.09703586 0.14209226 0.12652689 0.10348399]
Done!
(426, 2) (143, 2)
[[0.   1.  ]
 [0.   1.  ]
 [0.   1.  ]
 [1.   0.  ]
 [0.25 0.75]]
[[1.         0.        ]
 [0.18571427 0.81428564]
 [0.         1.        ]
 [0.05       0.95      ]
 [0.         1.        ]]


Сравним log_loss

In [83]:
clf_dt.fit(X,y)
pr=clf_dt.predict_proba(Xt)
print('DT: С использоваеним функции:', sklearn.metrics.log_loss(yt, sxt_dt))
print('DT: Без использования:',sklearn.metrics.log_loss(yt, pr))
print()

clf_lr.fit(X,y)
pr=clf_lr.predict_proba(Xt)
print('LR: С использоваеним функции:', sklearn.metrics.log_loss(yt, sxt_lr))
print('LR: Без использования:',sklearn.metrics.log_loss(yt, pr))

DT: С использоваеним функции: 0.08758764833837122
DT: Без использования: 1.0657655994422166

LR: С использоваеним функции: 0.10781664345334678
LR: Без использования: 0.10668813181332457
